<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sánchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matemático <br> 
    Instituto de Ingeniería Matemática y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Módulo 5: Mixed and HDG Methods</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->



In [1]:
from ngsolve import *
from ngsolve.webgui import Draw


## Model problem

Poisson equation 
\begin{equation}
-\nabla \cdot (c \nabla u) = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad c\nabla u \cdot n = g, \text{ on }\Gamma_N  
\end{equation}

In [31]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
source = sin(3.14*x)
ud = CF(5)
g = mesh.BoundaryCF( {"left" : y*(1-y)}, default=0)
c = 10

## Primal formulation and method

In [5]:
fesp = H1(mesh, order=4, dirichlet="bottom")
up, vp = fesp.TnT()

ap = BilinearForm(c*grad(up)*grad(vp)*dx).Assemble()
fp = LinearForm(source*vp*dx + g*vp * ds).Assemble()

gfup = GridFunction(fesp)
gfup.Set(ud, BND)

r = fp.vec - ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
Draw (-1*c* grad(gfup), mesh, "flux-primal");
Draw (gfup)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Mixed formulation and method
\begin{equation}
-\nabla \cdot (c \nabla u) = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad c\nabla u \cdot n = g, \text{ on }\Gamma_N  
\end{equation}

\begin{equation}
a q = \nabla u, \quad \nabla \cdot q = f, \quad u = u_{d}, \text{ on } \Gamma_{D}, \quad -q\cdot n = g, \text{ on }\Gamma_N  
\end{equation}

In [11]:
order_flux=1
V = HDiv(mesh, order=order_flux, dirichlet="right|top|left")
Q = L2(mesh, order=order_flux-1)
fesm = V*Q
q, u = fesm.TrialFunction()
r, v = fesm.TestFunction()
normal = specialcf.normal(mesh.dim)

afun = 1/c
am = BilinearForm((afun*q*r - div(q)*v - div(r)*u)*dx).Assemble()
fm = LinearForm(-source*v*dx - ud*(r.Trace()*normal)*ds).Assemble()

gfm = GridFunction(fesm)

gfq, gfu = gfm.components
gfq.Set(-g*normal, BND)
res = fm.vec.data - am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * res
# solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfq, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed");

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

## HDG formulation and method 

$$ \widehat{q}_h\cdot n = q_h\cdot n + \tau(u_h - \widehat{u}_h)$$

\begin{align*}
\int_{K} a q_h \cdot r - \int_{K} u_h\nabla\cdot r +\int_{\partial K} \widehat{u}_h r\cdot n ds &= 0 \\
\int_{K} q_h \cdot\nabla v + \int_{\partial K} \widehat{q}_h\cdot n v ds &= -\int_{K} f v \\
\sum_{K}\int_{\partial K \backslash \Gamma_D} \widehat{q}_h\cdot n \mu & = \int_{\Gamma_N} g \mu ds \\
\int_{\Gamma_D} \widehat{u}_h \mu ds & = \int_{Gamma_D} u_d \mu ds
\end{align*}

In [33]:
p=6; condense=True
V = VectorL2(mesh, order=p)
W = L2(mesh, order=p)
M = FacetFESpace(mesh, order=p, dirichlet='bottom')
feshdg = V*W*M
q, u, uhat = feshdg.TrialFunction()
r, v, mu   = feshdg.TestFunction()
normal = specialcf.normal(mesh.dim)
# stabilization parameter
tau = 1.0
ahdg = BilinearForm(feshdg, condense=condense)
ahdg += (afun*q*r - u*div(r))*dx + uhat*r*normal*dx(element_boundary=True)
ahdg += -div(q)*v*dx + (u-uhat)*v*dx(element_boundary=True)
ahdg += (q*normal+tau*(u-uhat))*mu*dx(element_boundary=True)
pre = Preconditioner(ahdg, "bddc", coarsetype="h1amg")

ahdg.Assemble()
fhdg = LinearForm(-source*v*dx - g*mu.Trace()*ds).Assemble()


gfhdg = GridFunction(feshdg)
gfq, gfu, gfuhat = gfhdg.components
gfuhat.Set(ud, BND)

# solvers.BVP(bf=ahdg, lf=fhdg, gf=gfhdg, pre=pre)

fmod = (fhdg.vec + ahdg.harmonic_extension_trans * fhdg.vec).Evaluate()
solvers.CG(mat=ahdg.mat, pre=pre.mat, rhs=fmod, sol=gfhdg.vec, initialize=False)
gfhdg.vec.data += ahdg.harmonic_extension * gfhdg.vec
gfhdg.vec.data += ahdg.inner_solve * fhdg.vec


# solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfq, mesh, "flux-hdg")
Draw (gfu, mesh, "u-hdg");

CG iteration 1, residual = 844.5383359807186     
CG iteration 2, residual = 305.61915847527973     
CG iteration 3, residual = 144.2102421160451     
CG iteration 4, residual = 141.17929102007065     
CG iteration 5, residual = 48.28657278051534     
CG iteration 6, residual = 32.36851128508233     
CG iteration 7, residual = 35.53779638150786     
CG iteration 8, residual = 22.750640545369286     
CG iteration 9, residual = 15.412567712583922     
CG iteration 10, residual = 18.45094205093878     
CG iteration 11, residual = 11.815672067262154     
CG iteration 12, residual = 9.322418785799579     
CG iteration 13, residual = 10.862639139197134     
CG iteration 14, residual = 7.587991949533158     
CG iteration 15, residual = 7.462009806048984     
CG iteration 16, residual = 7.095758336585243     
CG iteration 17, residual = 5.5264471845176     
CG iteration 18, residual = 3.8553624119084016     
CG iteration 19, residual = 3.9912870745137674     
CG iteration 20, residual = 3.7797

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…